# Project 3: Using Non-SQL Database to Execute the Vision

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Students: 

Edgar Leon, 

Javier Rodriguez, 

Melia Soque, 

Neha Jakkinpali

Year: 2023

Semester:Spring

Section:10


# Project 3 Data Substructure and Map Layers

## imports

In [18]:
import math
import numpy as np
import pandas as pd
import random

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

## my_select_query_pandas() - function to run a select query and return rows in a Pandas dataframe

In [19]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

## Connect to the Postgres database

In [20]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Create a cursor for the connection

In [21]:
cursor = connection.cursor()

## Connect to Google Maps using your api key;  edit the file gmap_api_key.txt and put in your api key

In [22]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

## Basic map centered on Sather Gate at UC Berkeley; all point are (latitude, longitude) in decimal

In [23]:
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)

# gmaps.figure(center=sather_gate_berkeley, zoom_level=9)

# Beggining of Project 3 code 

## For each customer, the addresses are fake, and the lat/lon is just the center of the zipcode, to generate random locations to use in our exercise, we will add a random number to each customer lat/lon entry

### From the Project Suggestions:  If you were wanting to generate some random locations, here is a suggestion:

The zip codes table has the latitude and longitude for the geographic center of the zip code, use this as a starting value
Take the latitude and generate a plus/minus variance using the Python random methods
Add the plus/minus variance to the latitude
Likewise, take the longitude, generate a plus/minus variance, and add the plus/minus variance to the longtitude
Now you have a random point


### First lets check the zip_codes table

In [7]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * from zip_codes



"""
df2 = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df2

,zip,latitude,longitude,city,state,population,area,density,time_zone
0,08074,39.7158,-75.1640,Richwood,NJ,15,0.0886,169.39,America/New_York
1,08240,39.4873,-74.5318,Pomona,NJ,2293,1.5196,1508.93,America/New_York
2,08876,40.5880,-74.6874,Somerville,NJ,22059,15.1172,1459.20,America/New_York
3,10001,40.7506,-73.9972,New York,NY,22924,0.6675,34341.44,America/New_York
4,32026,30.0541,-82.1815,Raiford,FL,1907,0.6333,3011.38,America/New_York
...,...,...,...,...,...,...,...,...,...
32718,47367,40.0827,-85.3872,Oakville,IN,23,0.0866,265.47,America/Indiana/Indianapolis
32719,63079,38.2606,-91.0998,Stanton,MO,24,0.3523,68.12,America/Chicago
32720,63738,37.0893,-89.9574,Brownwood,MO,31,0.1171,264.70,America/Chicago
32721,68954,40.6227,-98.2374,Inland,NE,14,1.7437,8.03,America/Chicago


### Lets check the customers table
The street addresses do not exist in the real world

In [7]:
query = """

select * from customers



"""
df3 = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df3

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1
...,...,...,...,...,...,...,...,...,...
31077,31078,Hugo,Domeney,529 5th Plaza,Thompsons Station,TN,37179,5,25
31078,31079,Glenn,Putson,1347 Westend Crossing,Thompsons Station,TN,37179,5,25
31079,31080,Minnie,Antham,9 Judy Place,Thompsons Station,TN,37179,5,25
31080,31081,Linet,Djorvic,29 Trailsway Drive,Thompsons Station,TN,37179,5,25


### Lets review and create the joint table 
#### The distance column is from the zip code lat/lon, therefore after adding the random locations, it will be invalid, but close
#### We may be able to get rid of the distance table, by substituting it with a real calculation using google map api
#### How much randomness to add to stay within the zip code boundary? staying within zip code boundary does not matter
#### What if they land on the water? Lets fix it later

In [8]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select z.zip, z.population, z.area, z.density, z.latitude, z.longitude,
cu.customer_id, cu.first_name, cu.last_name, cu.street, cu.city, cu.state, cu.closest_store_id, cu.distance
from customers as cu
     join zip_codes as z
         on cu.zip = z.zip
where cu.closest_store_id = 1
order by 1,2

"""

df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
df

,zip,population,area,density,latitude,longitude,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,94002,27202,5.9244,4591.53,37.5135,-122.2991,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,94002,27202,5.9244,4591.53,37.5135,-122.2991,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,94002,27202,5.9244,4591.53,37.5135,-122.2991,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,94002,27202,5.9244,4591.53,37.5135,-122.2991,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,94005,4692,4.8168,974.09,37.6887,-122.4080,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,94973,1342,3.7411,358.71,38.0127,-122.6397,8098,Luisa,Symcoxe,291 Thompson Street,Woodacre,CA,1,23
8134,94973,1342,3.7411,358.71,38.0127,-122.6397,8099,Adella,Buscher,20 Scofield Parkway,Woodacre,CA,1,23
8135,94973,1342,3.7411,358.71,38.0127,-122.6397,8094,Raviv,Pierton,5739 Holmberg Point,Woodacre,CA,1,23
8136,94973,1342,3.7411,358.71,38.0127,-122.6397,8093,Lemmy,Ettridge,24 Dahle Circle,Woodacre,CA,1,23


### Lets create a join table, and save the file as CSV, somewhere, 
#### One degree of latitude equals approximately 364,000 feet (69 miles), one minute equals 6,068 feet (1.15 miles), and one-second equals 101 feet. 
#### One-degree of longitude equals 288,200 feet (54.6 miles), one minute equals 4,800 feet (0.91 mile), and one second equals 80 feet.

#### Lets create the random point and add a derived column to the table, at random between 0 to 5 miles, or 0 to 5 minutes, both lat/lon

#### 60 minutes equals one degree of lat/lon, therefore 5 minutes is equal to 5/60, or 0.08333
#### lets try to generate a value between -0.10 and 0.10
#### The data is in the DF database, and we can do the random addresses with python
#### To make it more realistic lets use a uniform distribution, centered around 0 with std of 0.10 degrees (~ 1 mile)
#### A uniform distribution resembles houses distributed uniformly in a block 



In [14]:
#rollback_before_flag = True
#rollback_after_flag = True
import random
#n_lat stand for new latitude
#df.insert(6, 'n_lat', df['latitude'] + np.random.uniform(-0.08333, 0.08333)) 

df.insert(6, 'n_lat', df['latitude'] )
df.head()

,zip,population,area,density,latitude,longitude,n_lat,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.5135,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.5135,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.5135,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.5135,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,94005,4692,4.8168,974.09,37.6887,-122.4080,37.6887,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14


### Create a random latitude value 

In [12]:
for ind in df.index:
    #df['n_lat'][ind] = df['n_lat'][ind] + np.random.uniform(-0.00333, 0.00333)
    df['n_lat'][ind] = df['n_lat'][ind] + np.random.normal(0, 0.00633)
    
#for ind in df.index:
    #print(df['n_lat'][ind] )

df

/tmp/ipykernel_141/3471726429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_lat'][ind] = df['n_lat'][ind] + np.random.normal(0, 0.00633)


,zip,population,area,density,latitude,longitude,n_lat,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.520606,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.513840,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.510353,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.514176,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,94005,4692,4.8168,974.09,37.6887,-122.4080,37.682348,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,94973,1342,3.7411,358.71,38.0127,-122.6397,38.006360,8098,Luisa,Symcoxe,291 Thompson Street,Woodacre,CA,1,23
8134,94973,1342,3.7411,358.71,38.0127,-122.6397,38.017764,8099,Adella,Buscher,20 Scofield Parkway,Woodacre,CA,1,23
8135,94973,1342,3.7411,358.71,38.0127,-122.6397,38.003338,8094,Raviv,Pierton,5739 Holmberg Point,Woodacre,CA,1,23
8136,94973,1342,3.7411,358.71,38.0127,-122.6397,38.011278,8093,Lemmy,Ettridge,24 Dahle Circle,Woodacre,CA,1,23


In [13]:
#rollback_before_flag = True
#rollback_after_flag = True
import random
#n_lon stand for new longitude
df.insert(7, 'n_lon', df['longitude'])



df.head()

,zip,population,area,density,latitude,longitude,n_lat,n_lon,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.520606,-122.2991,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.513840,-122.2991,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.510353,-122.2991,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.514176,-122.2991,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,94005,4692,4.8168,974.09,37.6887,-122.4080,37.682348,-122.4080,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14


### Create the random longitude value

In [14]:
for ind in df.index:
    #df['n_lon'][ind] = df['n_lon'][ind] + np.random.uniform(-0.00333, 0.00333)
    df['n_lon'][ind] = df['n_lon'][ind] + np.random.normal(0, 0.00633)
    
df.head()

/tmp/ipykernel_141/1735542256.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_lon'][ind] = df['n_lon'][ind] + np.random.normal(0, 0.00633)


,zip,population,area,density,latitude,longitude,n_lat,n_lon,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.520606,-122.298226,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.513840,-122.289873,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.510353,-122.301011,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.514176,-122.306404,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,94005,4692,4.8168,974.09,37.6887,-122.4080,37.682348,-122.406707,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14


### Lets export the df into csv file


In [ ]:

import csv
df.to_csv("cusotmers_w_fake_addresses.csv")


## Import CSV file with customers with generated fake addresses

In [24]:
df = pd.read_csv('cusotmers_w_fake_addresses.csv')
df

,Unnamed: 0,zip,population,area,density,latitude,longitude,n_lat,n_lon,customer_id,first_name,last_name,street,city,state,closest_store_id,distance
0,0,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.520606,-122.298226,8103,Karil,Gives,458 Welch Avenue,Belmont,CA,1,24
1,1,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.513840,-122.289873,8100,Mercy,Blunsden,173 Fairview Pass,Belmont,CA,1,24
2,2,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.510353,-122.301011,8101,Levey,Duval,512 Melby Hill,Belmont,CA,1,24
3,3,94002,27202,5.9244,4591.53,37.5135,-122.2991,37.514176,-122.306404,8102,Margot,MacGiany,12 Fordem Crossing,Belmont,CA,1,24
4,4,94005,4692,4.8168,974.09,37.6887,-122.4080,37.682348,-122.406707,7158,Malissa,Latek,8346 Fieldstone Park,Brisbane,CA,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8133,8133,94973,1342,3.7411,358.71,38.0127,-122.6397,38.006360,-122.638831,8098,Luisa,Symcoxe,291 Thompson Street,Woodacre,CA,1,23
8134,8134,94973,1342,3.7411,358.71,38.0127,-122.6397,38.017764,-122.637123,8099,Adella,Buscher,20 Scofield Parkway,Woodacre,CA,1,23
8135,8135,94973,1342,3.7411,358.71,38.0127,-122.6397,38.003338,-122.627826,8094,Raviv,Pierton,5739 Holmberg Point,Woodacre,CA,1,23
8136,8136,94973,1342,3.7411,358.71,38.0127,-122.6397,38.011278,-122.641591,8093,Lemmy,Ettridge,24 Dahle Circle,Woodacre,CA,1,23


## Heatmap showing intensity as the number of customers for the Berkely store


In [27]:
fig = gmaps.figure(center=sather_gate_berkeley,zoom_level=9)

#heatmap_layer = gmaps.heatmap_layer(df)
heatmap_layer = gmaps.heatmap_layer(df[['n_lat','n_lon']])

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Overlay of BART stations with Interactive Markers



In [10]:
query = """

select * from stations



"""
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

info_box_template = """
<dl>
<dt>Station</dt><dd>{station}</dd>
</dl>
"""

df_list_dict = stations_df.to_dict('records')
df_list_dict



station_info = [info_box_template.format(**stations) for stations in df_list_dict]

marker_layer = gmaps.marker_layer(stations_df[['latitude','longitude']], info_box_content=station_info)

fig = gmaps.figure(center=sather_gate_berkeley, zoom_level=9)

fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

## Overlay of BART stations with Plain Dots

In [126]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select * from stations



"""
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)


#stations_df.head()
#print(station_locations )

#color green is chosen for dollars sign $$$

fig = gmaps.figure(center=sather_gate_berkeley, map_type = 'HYBRID', zoom_level=9)

df_markers = stations_df[['latitude','longitude']]

marker_layer = gmaps.symbol_layer(df_markers, fill_color='white', stroke_color='white' )

fig.add_layer(marker_layer)

fig


Figure(layout=FigureLayout(height='420px'))

## Overlay of BART stations and lines using geojson file 


In [25]:
#BART_Track_edited10.geojson is the json file that is well constructed
with open('BART_Track_edited10.geojson') as f:
    geometry = json.load(f)

fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(geometry)
fig.add_layer(geojson_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Overlay of BART track lines in black, BART stations with black dots, customer population heat map, and Meal Processing Center Location with an Intertactive Marker

In [26]:
rollback_before_flag = True
rollback_after_flag = True
query = """
select * from stations
"""
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

colors = []
for i in range(0,50):
    colors.append(np.random.choice(['black', 'blue', 'purple', 'yellow', 'teal', 'white', 'green', 'gray']))


fig = gmaps.figure(center=sather_gate_berkeley, zoom_level=9)

# map_type = 'HYBRID',

heatmap_layer = gmaps.heatmap_layer(df[['n_lat','n_lon']])
#heatmap_layer.max_intensity = 100
#heatmap_layer.point_radius = 5
df_markers = stations_df[['latitude','longitude']]
marker_layer = gmaps.symbol_layer(df_markers, fill_color=colors, stroke_color=colors )

fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)


#define meal processing center
d = {'name': ['Meal Processing Center'],
'latitude': [37.390521000000014],
'longitude': [-121.89486009051173]
    }
mcenter_df = pd.DataFrame(d)
meal_processing_center_location = mcenter_df[['latitude','longitude']]

info_box_template = """
<dl>
<dt>Location</dt><dd>{name}</dd>
</dl>
"""
df_list_dict = mcenter_df.to_dict('records')
df_list_dict

station_info = [info_box_template.format(**stations) for stations in df_list_dict]
marker_layer = gmaps.marker_layer(meal_processing_center_location, info_box_content=station_info)
fig.add_layer(marker_layer)


with open('BART_Track_edited10.geojson') as f:
    geometry = json.load(f)

#fig = gmaps.figure()
geojson_layer = gmaps.geojson_layer(geometry, stroke_color ='purple')
fig.add_layer(geojson_layer)




#display all
fig


Figure(layout=FigureLayout(height='420px'))


## Get Driving Directions for Delivery Truck
### From Meal Processing Center to a BART Station
#### in this example we are using Warm Springs

In [17]:
query = """
select * from stations
"""
stations_df = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

#code to calculate driving directions from meal processing center to dublin

destination= stations_df.loc[stations_df ['station'] == 'Dublin']

#meal_processing_center_location is a dataframe therefore convert to tuple, o as in origin
origin = tuple(meal_processing_center_location.to_records(index=False)[0])

#convert to tuple, as well
destination= (destination.to_records(index = False)[0][1], destination.to_records(index = False)[0][2])

#midpoint is the middle between origin and destination
midpoint = ((origin[0]+destination[0])/2, (origin[1]+destination[1])/2)

fig = gmaps.figure(center=midpoint, zoom_level=11)

sather_2_chm = gmaps.directions_layer(origin, destination)
fig.add_layer(sather_2_chm)

fig

Figure(layout=FigureLayout(height='420px'))

# Community Detection: move over to Project_3-2_Neo4J_section.ipynb

